In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [71]:
df=pd.read_csv(r"C:\Users\Harshit\OneDrive\Desktop\GAME STREM REVIEW\DATA\output.csv")


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201151 entries, 0 to 201150
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           201151 non-null  int64 
 1   app_id       201151 non-null  int64 
 2   content      200723 non-null  object
 3   author_id    201151 non-null  int64 
 4   is_positive  201151 non-null  object
dtypes: int64(3), object(2)
memory usage: 7.7+ MB


In [73]:
df=df.rename(columns={"is_positive":"labels"})


In [74]:
df.isnull().sum()

id             0
app_id         0
content      428
author_id      0
labels         0
dtype: int64

In [75]:
df.dropna(subset=["content"],inplace=True)
df.isnull().sum()

id           0
app_id       0
content      0
author_id    0
labels       0
dtype: int64

In [76]:
df["content_length"]=df["content"].apply(len)

In [77]:
df["content_length"].describe()

count    200723.000000
mean        143.123035
std        1044.805167
min           1.000000
25%          15.000000
50%          41.000000
75%         115.000000
max      419580.000000
Name: content_length, dtype: float64

In [78]:
df=df.drop(df[df["content_length"]>100000].index)

In [79]:
df.groupby("labels").describe()

id                                                    \
             count          mean           std     min           25%   
labels                                                                 
Negative   98344.0  1.301288e+08  5.452696e+07  2506.0  1.008069e+08   
Positive  102377.0  1.150236e+08  6.206512e+07   209.0  5.704953e+07   

                                                   app_id              ...  \
                  50%          75%          max     count        mean  ...   
labels                                                                 ...   
Negative  162122664.0  167608236.5  182234883.0   98344.0  560.949972  ...   
Positive  136933742.0  171453085.0  182235002.0  102377.0  349.205759  ...   

             author_id               content_length                          \
                   75%           max          count        mean         std   
labels                                                                        
Negative  7.656120e+16  7.656120e+16        98344.0  165.464370  422.446753   
Positive  7.656120e+16  7.656120e+16       102377.0  116.394083  329.960921   

                                           
          min   25%   50%     75%     max  
labels                                     
Negative  1.0  17.0  49.0  143.25  8000.0  
Positive  1.0  13.0  35.0   92.00  8000.0  

[2 rows x 32 columns]

In [80]:
df.head()

,id,app_id,content,author_id,labels,content_length
0,181331361,100,At least its a counter strike -1/100,76561199556485100,Negative,36
1,180872601,100,Uh... So far my playthrough has not been great...,76561199230620391,Negative,230
2,177836246,100,Better mechanics than cs2,76561198417690647,Negative,25
3,177287444,100,buggy mess and NOT fun to play at all,76561199077268730,Negative,37
4,176678990,100,"Whoever came up with this, is gonna fucking ge...",76561199104544266,Negative,76


In [91]:
df["content"][0]

'At least its a counter strike -1/100'

TEXT PRE PROCESSING

In [81]:
def text_pre_process(text):
    non_punch=[c for c in text if c not in string.punctuation]
    non_punch="".join(non_punch)

    return[word for word in non_punch.split() if word.lower() not in stopwords.words("english")]

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier

In [100]:
text_train,text_test,label_train,label_test=train_test_split(df["content"],df["labels"],test_size=0.3)

In [101]:
pipeline=Pipeline([
    ("bow",CountVectorizer(analyzer=text_pre_process)),
    ("tf_idf_transformer",TfidfTransformer()),
    ("classification",PassiveAggressiveClassifier(max_iter=50, random_state=0))
])

In [102]:
pipeline.fit(text_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_pre_process at 0x000001F60017E5E0>)),
                ('tf_idf_transformer', TfidfTransformer()),
                ('classification',
                 PassiveAggressiveClassifier(max_iter=50, random_state=0))])

In [103]:
pred=pipeline.predict(text_test)

In [104]:
from sklearn.metrics import classification_report,confusion_matrix

In [105]:
print(classification_report(pred,label_test))

              precision    recall  f1-score   support

    Negative       0.79      0.81      0.80     28636
    Positive       0.83      0.80      0.82     31581

    accuracy                           0.81     60217
   macro avg       0.81      0.81      0.81     60217
weighted avg       0.81      0.81      0.81     60217



In [106]:
print(confusion_matrix(pred,label_test))

[[23313  5323]
 [ 6199 25382]]


In [90]:
import pickle
with open("review_model","wb") as file:
    pickle.dump(pipeline,file)
    print("model saved sucessfully")

model saved sucessfully


In [107]:
pipeline.predict(["cancer"])

array(['Negative'], dtype='<U8')